<a href="https://colab.research.google.com/github/MZiaAfzal71/Edge-Aware-GNN/blob/main/Models/Edge_Aware_GNN_for_ESOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Edge-Aware GNN for ESOL

This notebook implements an **Edge-Aware Graph Neural Network (EA-GNN)** for predicting
aqueous solubility on the **ESOL (Delaney) dataset**.

The model represents each molecule as a graph derived from its **SMILES string**, where:
- **Nodes** correspond to atoms with chemically meaningful atom features
- **Edges** correspond to bonds with explicit bond features
- Message passing is performed using **edge-aware GNN layers** to incorporate bond information

In this notebook:
- Only **SMILES-based molecular graphs** are used
- **No global physicochemical descriptors** are included
- The model relies solely on **atom- and bond-level information**
- Training and evaluation are performed using:
  - **Repeated cross-validation**, and
  - A **Bemis–Murcko scaffold-based split** to assess generalization to unseen scaffolds

This setup allows us to assess the predictive capability of an edge-aware GNN
when learning molecular representations directly from chemical structure alone,
both under random sampling and chemically realistic scaffold-based evaluation.

The notebook is designed to be:
- **Reproducible**
- **Interpretable**
- **Focused on structure–property relationships**

The ESOL dataset contains 1,128 small molecules with experimentally measured
aqueous solubility values, making it a standard benchmark for molecular property
prediction using graph neural networks.


In [ ]:
# 1️⃣ Fetch data
!git clone https://github.com/MZiaAfzal71/Edge-Aware-GNN.git

In [ ]:
# 2️⃣ Change current/working directory
%cd Combined-Interaction-Matrix/ESOL\ Dataset

In [ ]:
# 3️⃣ Install rdkit and PyG
!pip install rdkit torch_geometric

In [ ]:
#  4️⃣ Imports
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import random
import copy
import os

import torch
from torch_geometric.data import Data
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINEConv, AttentionalAggregation
import torch.optim as optim


from rdkit import Chem
# from rdkit.Chem.Scaffolds import MurckoScaffold

from sklearn.model_selection import RepeatedKFold
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
# 5️⃣ Set random seeds for reproducibility across Python, NumPy, and PyTorch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
# 6️⃣ Utility functions to convert SMILES strings into graph representations with atom and bond features

ELECTRONEGATIVITY = {
    1: 2.20, 6: 2.55, 7: 3.04, 8: 3.44,
    9: 3.98, 15: 2.19, 16: 2.58,
    17: 3.16, 35: 2.96, 53: 2.66
}

def atom_features(atom):
    Z = atom.GetAtomicNum()

    hyb = atom.GetHybridization()
    hyb_onehot = [
        hyb == Chem.rdchem.HybridizationType.SP,
        hyb == Chem.rdchem.HybridizationType.SP2,
        hyb == Chem.rdchem.HybridizationType.SP3
    ]

    chiral = atom.GetChiralTag()
    chiral_onehot = [
        chiral == Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW,
        chiral == Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CCW
    ]

    return [
        Z,
        # atom.GetMass(),
        atom.GetDegree(),
        atom.GetTotalValence(),
        atom.GetTotalNumHs(),
        atom.GetFormalCharge(),
        float(atom.GetIsAromatic()),
        float(atom.IsInRing()),
        *hyb_onehot,
        # *chiral_onehot,
        ELECTRONEGATIVITY.get(Z, 0.0)
    ]

def bond_features(bond):
    bt = bond.GetBondType()

    stereo = bond.GetStereo()
    stereo_onehot = [
        stereo == Chem.rdchem.BondStereo.STEREOE,
        stereo == Chem.rdchem.BondStereo.STEREOZ
    ]

    return [
        bt == Chem.rdchem.BondType.SINGLE,
        bt == Chem.rdchem.BondType.DOUBLE,
        bt == Chem.rdchem.BondType.TRIPLE,
        bt == Chem.rdchem.BondType.AROMATIC,
        float(bond.GetIsConjugated()),
        float(bond.IsInRing()),
        # float(bond.IsRotor()),
        # float(bond.GetIsAmide()),
        # *stereo_onehot
    ]

def smiles_to_graph(smiles, y=None):
    mol = Chem.MolFromSmiles(smiles)

    x = [atom_features(atom) for atom in mol.GetAtoms()]

    edge_index, edge_attr = [], []

    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        bf = bond_features(bond)

        edge_index += [[i, j], [j, i]]
        edge_attr += [bf, bf]

    data = Data(
        x=torch.tensor(x, dtype=torch.float),
        edge_index=torch.tensor(edge_index, dtype=torch.long).t().contiguous(),
        edge_attr=torch.tensor(edge_attr, dtype=torch.float)
    )

    if y is not None:
        data.y = torch.tensor(y, dtype=torch.float)

    return data

In [ ]:
# 7️⃣ PyTorch Dataset Class

class MoleculeDataset(Dataset):
    def __init__(self, df, smiles_col, target_col):
        self.smiles = df[smiles_col].values
        self.targets = df[target_col].values.astype(np.float32)

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        graph = smiles_to_graph(self.smiles[idx])
        graph.y = torch.tensor([self.targets[idx]], dtype=torch.float)

        return graph


In [ ]:
# 8️⃣ GINE-based graph neural network with attentional readout for molecular graphs

class SimpleGINE(nn.Module):
    def __init__(
        self,
        node_dim,
        edge_dim,
        hidden_dim=128,
        dropout=0.1
    ):
        super().__init__()

        def make_mlp():
            return nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_dim),
                nn.Linear(hidden_dim, hidden_dim)
            )

        self.node_emb = nn.Linear(node_dim, hidden_dim)

        self.conv1 = GINEConv(make_mlp(), edge_dim=edge_dim)
        self.conv2 = GINEConv(make_mlp(), edge_dim=edge_dim)
        self.conv3 = GINEConv(make_mlp(), edge_dim=edge_dim)

        # ---- Attention readout ----
        self.readout = AttentionalAggregation(
            gate_nn=nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            )
        )

        self.dropout = dropout

    def forward(self, x, edge_index, edge_attr, batch):
        x = self.node_emb(x)

        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = F.relu(self.conv3(x, edge_index, edge_attr))

        return self.readout(x, batch)

In [ ]:
# 9️⃣ Edge-aware graph neural network based on GINE for molecular property prediction

class Edge_aware_GNN(nn.Module):
    def __init__(
        self,
        node_dim,
        edge_dim,
        hidden_dim=128,
        dropout=0.1
    ):
        super().__init__()

        # ---- GNN backbone (unchanged) ----
        self.gnn = SimpleGINE(
            node_dim=node_dim,
            edge_dim=edge_dim,
            hidden_dim=hidden_dim,
            dropout=dropout
        )

        # ---- Prediction head (same as before) ----
        self.head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, graphs):
        g = self.gnn(
            graphs.x,
            graphs.edge_index,
            graphs.edge_attr,
            graphs.batch
        )

        return self.head(g).squeeze(-1)


In [ ]:
# 🔟 Utility function for repeated k-fold cross-validation with PyG data loaders

def run_repeated_kfold_cv(
    df,
    smiles_col="smiles",
    target_col="target",
    n_splits=5,
    n_repeats=5,
    batch_size=32,
    seed=42
):
    rkf = RepeatedKFold(
        n_splits=n_splits,
        n_repeats=n_repeats,
        random_state=seed
    )

    split_id = 0

    for train_idx, val_idx in rkf.split(df):
        repeat = split_id // n_splits
        fold   = split_id % n_splits

        train_df = df.iloc[train_idx]
        val_df   = df.iloc[val_idx]

        # ---- Train dataset (fit scaler) ----
        train_dataset = MoleculeDataset(
            train_df,
            smiles_col,
            target_col
        )

        # ---- Validation dataset (reuse scaler) ----
        val_dataset = MoleculeDataset(
            val_df,
            smiles_col,
            target_col
        )

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False
        )

        yield repeat, fold, train_loader, val_loader

        split_id += 1


In [ ]:
# 1️⃣1️⃣ # Training utility with early stopping, learning rate scheduling, and model checkpointing

class Trainer:
    def __init__(
        self,
        model,
        device,
        lr=1e-3,
        weight_decay=1e-4,
        patience=10,
        max_epochs=100
    ):
        self.model = model
        self.device = device
        self.patience = patience
        self.max_epochs = max_epochs

        self.criterion = nn.MSELoss()
        # self.criterion = nn.SmoothL1Loss(beta=1.0)

        self.optimizer = optim.Adam(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay
        )

        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode="min",
            factor=0.5,
            patience=5
        )

        self.best_val_loss = float("inf")
        self.best_model_state = None
        self.history = {
            "train_loss": [],
            "val_loss": []
        }


    def _train_one_epoch(self, train_loader):
        self.model.train()
        running_loss = 0.0

        for graphs in train_loader:
            graphs = graphs.to(self.device)

            self.optimizer.zero_grad()
            preds = self.model(graphs)
            loss = self.criterion(preds, graphs.y)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()

        return running_loss / len(train_loader)


    def _evaluate(self, loader):
        self.model.eval()
        total_loss = 0.0
        preds_all, targets_all = [], []

        with torch.no_grad():
            for graphs in loader:
                graphs = graphs.to(self.device)

                preds = self.model(graphs)
                loss = self.criterion(preds, graphs.y)

                total_loss += loss.item()
                preds_all.append(preds.cpu())
                targets_all.append(graphs.y.cpu())

        return (
            total_loss / len(loader),
            torch.cat(preds_all),
            torch.cat(targets_all)
        )


    def fit(self, train_loader, val_loader, verbose=True):
        patience_counter = 0

        for epoch in range(self.max_epochs):

            train_loss = self._train_one_epoch(train_loader)
            val_loss, _, _ = self._evaluate(val_loader)

            self.scheduler.step(val_loss)

            self.history["train_loss"].append(train_loss)
            self.history["val_loss"].append(val_loss)

            if verbose:
                print(
                    f"Epoch {epoch:03d} | "
                    f"Train: {train_loss:.4f} | "
                    f"Val: {val_loss:.4f}"
                )

            # ---- Best Model Tracking ----
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.best_model_state = copy.deepcopy(
                    self.model.state_dict()
                )
                patience_counter = 0
            else:
                patience_counter += 1

            # ---- Early Stopping ----
            if patience_counter >= self.patience:
                if verbose:
                    print("Early stopping triggered.")
                break

        # Restore best model
        self.model.load_state_dict(self.best_model_state)

        return self.model, self.best_val_loss


    def test(self, test_loader):
        return self._evaluate(test_loader)


    def save_best_model(self, path):
        torch.save(self.best_model_state, path)


    def load_model(self, path):
        self.model.load_state_dict(torch.load(path))


In [ ]:
# 1️⃣2️⃣ Load dataset, standardize target variable, and prepare data for modeling

file_path = "delaney-processed-scaffold.csv"
smiles_col = "smiles"
target_col = "measured log solubility in mols per litre"

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

df = pd.read_csv(file_path)

y = df[target_col]

y_mean = y.mean()
y_std  = y.std()

y_scaled = (y - y_mean) / y_std

df_data = pd.concat([df[smiles_col], y_scaled], axis=1)


In [ ]:
# 1️⃣2️⃣ Run repeated k-fold cross-validation training and record best validation loss per fold

fold_results = []

for repeat, fold, train_loader, val_loader in run_repeated_kfold_cv(
    df_data,
    smiles_col=smiles_col,
    target_col=target_col,
    n_splits=5,
    n_repeats=5
):
    print(f"\n===== Repeat {repeat + 1} | Fold {fold + 1} =====")

    model = Edge_aware_GNN(
        node_dim=11,
        edge_dim=6,
        hidden_dim=128,
        dropout=0.1
    ).to(device)

    trainer = Trainer(
        model=model,
        device=device,
        lr=1e-3,
        patience=20,
        max_epochs=150
    )

    model, best_val_loss = trainer.fit(
        train_loader,
        val_loader
    )

    _, train_preds, train_targets = trainer.test(train_loader)
    _, val_preds, val_targets = trainer.test(val_loader)

    train_preds_true = (train_preds * y_std + y_mean).numpy()
    train_targets_true = (train_targets * y_std + y_mean).numpy()

    val_preds_true = (val_preds * y_std + y_mean).numpy()
    val_targets_true = (val_targets * y_std + y_mean).numpy()

    train_rmse = np.sqrt(mean_squared_error(
        train_preds_true,
        train_targets_true
    ))

    train_r2 = r2_score(
        train_preds_true,
        train_targets_true
    )

    train_mae = mean_absolute_error(
        train_preds_true,
        train_targets_true
    )

    val_rmse = np.sqrt(mean_squared_error(
        val_preds_true,
        val_targets_true
    ))

    val_r2 = r2_score(
        val_preds_true,
        val_targets_true
    )

    val_mae = mean_absolute_error(
        val_preds_true,
        val_targets_true
    )

    fold_results.append({
        "repeat": repeat + 1,
        "fold": fold + 1,
        "best_train_rmse": train_rmse,
        "best_train_r2": train_r2,
        "best_train_mae": train_mae,
        "best_val_rmse": val_rmse,
        "best_val_r2": val_r2,
        "best_val_mae": val_mae
    })

    print(
        f"Repeat {repeat + 1} | Fold {fold + 1} | "
        f"Best Val Loss: {best_val_loss:.4f}"
    )

fold_results_df = pd.DataFrame(fold_results)
fold_results_df.to_csv("Fold Results Edge aware GNN.csv", index=False)


In [ ]:
# 1️⃣3️⃣ Train an ensemble of edge-aware GNN models using scaffold-based splits and report train/validation metrics

def train_ensemble_scaffold(
    df,
    split_col,
    device,
    smiles_col=smiles_col,
    target_col=target_col,
    y_mean=y_mean,
    y_std=y_std,
    batch_size=32,
    num_models=10,
    seed_start=42
):
    train_ind = split_col[split_col == "Train"].index
    val_ind = split_col[split_col != "Train"].index

    train_df = df.loc[train_ind]
    train_y_true = (df[target_col][train_ind] * y_std + y_mean).to_numpy()

    val_df = df.loc[val_ind]
    val_y_true = (df[target_col][val_ind] * y_std + y_mean).to_numpy()

    # ---- Train dataset (fit scaler) ----
    train_dataset = MoleculeDataset(
        train_df,
        smiles_col,
        target_col
    )

    # ---- Validation dataset (reuse scaler) ----
    val_dataset = MoleculeDataset(
        val_df,
        smiles_col,
        target_col
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )

    trained_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    fold_results = []

    for i in range(num_models):
        print(f"\n===== Ensemble model {i+1}/{num_models} =====")
        set_seed(seed_start + i)

        model = Edge_aware_GNN(
            node_dim=11,
            edge_dim=6,
            hidden_dim=128,
            dropout=0.1
        ).to(device)

        trainer = Trainer(
            model=model,
            device=device,
            lr=1e-3,
            patience=20,
            max_epochs=150
        )

        model, best_val_loss = trainer.fit(
            train_loader,
            val_loader
        )

        _, train_preds, _ = trainer.test(trained_loader)
        _, val_preds, _ = trainer.test(val_loader)

        train_preds_true = (train_preds * y_std + y_mean).numpy()

        val_preds_true = (val_preds * y_std + y_mean).numpy()

        train_rmse = np.sqrt(mean_squared_error(
            train_preds_true,
            train_y_true
        ))

        train_r2 = r2_score(
            train_preds_true,
            train_y_true
        )

        train_mae = mean_absolute_error(
            train_preds_true,
            train_y_true
        )

        val_rmse = np.sqrt(mean_squared_error(
            val_preds_true,
            val_y_true
        ))

        val_r2 = r2_score(
            val_preds_true,
            val_y_true
        )

        val_mae = mean_absolute_error(
            val_preds_true,
            val_y_true
        )

        fold_results.append({
            "Ensemble": i + 1,
            "best_train_rmse": train_rmse,
            "best_train_r2": train_r2,
            "best_train_mae": train_mae,
            "best_val_rmse": val_rmse,
            "best_val_r2": val_r2,
            "best_val_mae": val_mae
        })

        print(
            f"Ensemble {i + 1} | "
            f"Best Val Loss: {best_val_loss:.4f}"
        )


    return fold_results



In [ ]:
# 1️⃣4️⃣ Run scaffold-based ensemble training for the edge-aware GNN and save fold-wise performance metrics

results = train_ensemble_scaffold(
            df_data,
            df['BM-Scaffold'],
            device,
            smiles_col=smiles_col,
            target_col=target_col,
            y_mean=y_mean,
            y_std=y_std
        )
results_df = pd.DataFrame(results)
results_df.to_csv("Ensemble results edge aware GNN Scaffold.csv", index=False)

In [ ]:
# 1️⃣5️⃣

In [ ]:
# 1️⃣6️⃣

In [ ]:
# 1️⃣7️⃣

In [ ]:
# 1️⃣8️⃣

In [ ]:
# 1️⃣9️⃣